___
## Módulo 8: Temas avanzados.
___

## 8.2.  Web Scraping I

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

**8.2.1** De la página: https://www.expansion.com/mercados/cotizaciones/indices/ibex35_I.IB.html obten un daframe con los contenidos de la tabla de cotizaciones.

In [47]:
dfs = pd.read_html('https://www.expansion.com/mercados/cotizaciones/indices/ibex35_I.IB.html', match='Valor')

In [35]:
len(dfs)

5

In [37]:
dfs[0]

,Valor,Último,Var. %,Var.,Ac. % año,Máx.,Mín.,Vol.,Capit.,Hora,Unnamed: 10
0,ACCIONA,184300,126,230,721,184800,180800,38.611,10.110,16:31,NaN
1,ACCIONA ENER,35580,154,54,-155,35660,34900,138.596,11.715,16:35,NaN
2,ACERINOX,9484,83,8,571,9512,9390,227.704,2.566,16:36,NaN
3,ACS,29380,79,23,1166,29490,29180,252.886,8.349,16:36,NaN
4,AENA,148750,-13,-20,2681,149450,147000,45.057,22.312,16:36,NaN
5,AMADEUS IT GROUP,61600,155,94,2688,61640,60040,317.070,27.751,16:35,NaN
6,ARCELORMITTAL,27860,98,27,1330,27910,27460,130.784,24.456,16:36,NaN
7,BANCO SABADELL,992,-174,-2,1267,1016,986,14.750.958,5.584,16:36,NaN
8,BANKINTER,5240,-187,-10,-1493,5310,5156,3.156.694,4.710,16:36,NaN
9,BBVA,6594,-38,-3,1704,6695,6543,7.236.283,42.114,16:35,NaN


In [23]:
url = 'https://www.expansion.com/mercados/cotizaciones/indices/ibex35_I.IB.html'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [39]:
result = soup.find_all('table',{'id' : 'listado_valores'})

In [40]:
len(result)

1

In [44]:
pd.read_html(str(result[0]))[0]

,Valor,Último,Var. %,Var.,Ac. % año,Máx.,Mín.,Vol.,Capit.,Hora,Unnamed: 10
0,ACCIONA,184300,126,230,721,184800,180800,38.611,10.110,16:31,NaN
1,ACCIONA ENER,35600,160,56,-149,35660,34900,138.217,11.721,16:33,NaN
2,ACERINOX,9480,79,7,567,9512,9390,224.349,2.565,16:34,NaN
3,ACS,29390,82,24,1170,29490,29180,252.570,8.352,16:34,NaN
4,AENA,148950,0,0,2698,149450,147000,44.458,22.342,16:28,NaN
5,AMADEUS IT GROUP,61580,152,92,2684,61640,60040,315.395,27.742,16:34,NaN
6,ARCELORMITTAL,27865,100,27,1332,27910,27460,130.316,24.460,16:33,NaN
7,BANCO SABADELL,993,-168,-2,1274,1016,986,14.720.268,5.588,16:34,NaN
8,BANKINTER,5246,-176,-9,-1483,5310,5156,3.151.225,4.715,16:32,NaN
9,BBVA,6593,-39,-3,1702,6695,6543,7.189.112,42.107,16:34,NaN


**8.2.2** Extrae las noticias que aparecen el la web: https://www.expansion.com/mercados/cotizaciones/valores/telefonica_M.TEF.html, Generando un dataframe con la fecha, el título y el resumen de cada noticia.


In [57]:
url = 'https://www.expansion.com/mercados/cotizaciones/valores/telefonica_M.TEF.html'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [58]:
noticas = soup.find_all('article',{'class' : 'noticia'})

In [60]:
len(noticas)

10

In [82]:
list_noticias = []

for noticia in noticas:
    
    fecha = noticia.find_all('p',{'class' : 'fecha'})[0].text
    titulo = noticia.find_all('a',)[0].text
    entradilla = noticia.find_all('div',{'class' : 'entradilla'})[0].text

    list_noticias.append(
        {
            'fecha': fecha,
            'titulo': titulo,
            'entradilla': entradilla,
        }
    )

In [84]:
pd.DataFrame(list_noticias)

,fecha,titulo,entradilla
0,31-03-2023,"Pallete: ""Un nuevo mundo requiere nuevas reglas""","\nEl presidente de Telefónica, José María Alva..."
1,30-03-2023,La CNMC fija ahora el reparto del servicio uni...,\nLa CNMC acaba de anunciar como se debe repar...
2,28-03-2023,Impecable apoyo de Telefónica en los máximos d...,\nA pesar de las importantes caídas durante gr...
3,28-03-2023,El titán del Ibex que destaca entre sus rivale...,\nUna compañía del Ibex es una joya para la ma...
4,28-03-2023,Telefónica y Asterion cierran la venta de 11 c...,"\nNabiax, la firma conjunta de la 'teleco' y e..."
5,25-03-2023,Las cotizadas disparan un 28% sus ingresos has...,\nEl ebitda conjunto crece un 9%. El beneficio...
6,21-03-2023,Movistar Money abre sus créditos a clientes de...,\n\n
7,21-03-2023,"JPMorgan augura rumbos dispares en Iberdrola, ...",\nLas aguas podrían tardar en volver a su cauc...
8,16-03-2023,Movistar Prosegur Alarmas prevé que el mercado...,"\nMovistar Prosegur Alarmas (MPA), la joint ve..."
9,14-03-2023,"Telefónica, en soporte, consigue aguantar el t...",\nLos títulos de nuestra operadora cayeron aye...


##  8.3. Web Scraping II.
**8.3.1** Navega hasta la página Mercados y cotizaciones, apartado precios. Y descarga la tabla con los precios del ibex. Puedes obtener el código html de la tabla con .get_attribute('innerHTML') y utiliza pandas pd.read_html para sacarla los resultados en un dataframe.

In [2]:
from selenium import webdriver

DRIVER_PATH = '../module_8//chromedriver' # El mío está en la carpeta del módulo 8
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://www.bolsamadrid.es/')

/var/folders/p5/_qx9l9gs30s9_8y4qcb7jkwm0000gn/T/ipykernel_59330/834565130.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
from selenium.webdriver.common.by import By

In [4]:
botton_cookies = driver.find_element(
    By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
)

In [5]:
botton_cookies.text

'Aceptar todas las cookies'

In [6]:
botton_cookies.click()

In [7]:
botton_mercados = driver.find_element(
    By.XPATH, '//*[@id="navbar-menu"]/li[4]/a'
)

In [8]:
botton_mercados.text

'Mercados y Cotizaciones'

In [9]:
botton_mercados.click()

In [10]:
botton_precios = driver.find_element(
    By.XPATH, '//*[@id="menu135"]/ul/li[1]/ul/li[1]/a'
)

In [11]:
botton_precios.text

'Precios'

In [12]:
botton_precios.click()

In [13]:
tabla = driver.find_element(
    By.XPATH, '//*[@id="root"]/div[4]'
)

In [14]:
import pandas as pd

In [15]:
pd.read_html(tabla.get_attribute('innerHTML'))[0]

,Nombre,Último,% Dif.,Máximo,Mínimo,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,1805000,"1,58%",1823000,1794500,38.343,"6.940,57",14/04/2023,13:12:29
1,ACCIONA ENER,347000,"-0,17%",351800,347000,52.932,"1.845,49",14/04/2023,13:07:38
2,ACERINOX,95040,"0,23%",95140,94400,186.650,"1.768,97",14/04/2023,13:11:58
3,ACS,304500,"0,96%",304700,300500,137.690,"4.173,87",14/04/2023,13:12:01
4,AENA,1508000,"-0,07%",1516500,1496500,19.074,"2.869,53",14/04/2023,13:13:28
5,AMADEUS,610800,"0,13%",612800,609000,84.369,"5.158,22",14/04/2023,13:13:54
6,ARCELORMIT.,269150,"-0,55%",272200,267600,49.528,"1.332,33",14/04/2023,13:11:55
7,B.SANTANDER,35500,"2,03%",35560,34745,10.200.269,"35.584,93",14/04/2023,13:14:03
8,BA.SABADELL,10240,"2,52%",10245,9878,10.149.094,"10.188,36",14/04/2023,13:13:57
9,BANKINTER,56040,"3,01%",56060,54320,920.128,"5.062,83",14/04/2023,13:14:00


**8.3.2** Obtén de Investing.com la información histórica del Banco Santander, desde el 1 de enero del 2018, hasta el 31 de diciembre del mismo año

https://es.investing.com/equities/banco-santander-historical-data

In [ ]:
DRIVER_PATH = 'chromedriver' # El mío está en la carpeta del módulo 8
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://es.investing.com/equities/banco-santander-historical-data')

Ojo, porque al acceder a investing desde código, lo primero que nos encontraremos serán 2 pop up

<center>
<img src="./entrada_investing.png"  alt="drawing" width="600"/>
</center>

In [ ]:
boton_aceptar = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
boton_aceptar.click()

In [ ]:
boton_cerrar = driver.find_element_by_xpath('//*[@id="PromoteSignUpPopUp"]/div[2]/i')
boton_cerrar.click()

In [ ]:
boton_acciones = driver.find_element_by_xpath('//*[@id="widgetFieldDateRange"]')
boton_acciones.click()

In [ ]:
fecha_inicio = driver.find_element_by_xpath('//*[@id="startDate"]')
fecha_inicio.clear()
fecha_inicio.send_keys('01/01/2018')

In [ ]:
fecha_fin = driver.find_element_by_xpath('//*[@id="endDate"]')
fecha_fin.clear()
fecha_fin.send_keys('31/12/2018')

In [ ]:
boton_aceptar = driver.find_element_by_xpath('//*[@id="applyBtn"]')
boton_aceptar.click()

In [ ]:
tabla = driver.find_element_by_xpath('//*[@id="results_box"]')
tabla_html = tabla.get_attribute('innerHTML')
df = pd.read_html(tabla_html)
df[0]